<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Visualizing-Labels-produced-by-Dandelion,-using-fbTREX-semantic-API." data-toc-modified-id="Visualizing-Labels-produced-by-Dandelion,-using-fbTREX-semantic-API.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Visualizing Labels produced by Dandelion, using fbTREX semantic API.</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#First-of-all,-let's-combine-the-data-the-data-for-two-users-and-take-a-look-at-the-dataset." data-toc-modified-id="First-of-all,-let's-combine-the-data-the-data-for-two-users-and-take-a-look-at-the-dataset.-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>First of all, let's combine the data the data for two users and take a look at the dataset.</a></span></li><li><span><a href="#The-we-group-by-word-and-get-the-n-top-labels-for-each-user,-each-day." data-toc-modified-id="The-we-group-by-word-and-get-the-n-top-labels-for-each-user,-each-day.-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>The we group by word and get the n top labels for each user, each day.</a></span></li><li><span><a href="#Panoramic-of-top-labels-per-user." data-toc-modified-id="Panoramic-of-top-labels-per-user.-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Panoramic of top labels per user.</a></span></li><li><span><a href="#Compare-how-specific-words-have-been-appearing-on-two-different-users-timelines." data-toc-modified-id="Compare-how-specific-words-have-been-appearing-on-two-different-users-timelines.-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>Compare how specific words have been appearing on two different users timelines.</a></span></li><li><span><a href="#At-an-higher-level,-we-can-also-just-compare-the-top-contents-of-the-two-users-Newsfeed." data-toc-modified-id="At-an-higher-level,-we-can-also-just-compare-the-top-contents-of-the-two-users-Newsfeed.-1.0.5"><span class="toc-item-num">1.0.5&nbsp;&nbsp;</span>At an higher level, we can also just compare the top contents of the two users Newsfeed.</a></span></li></ul></li><li><span><a href="#You-can-also-make-a-Dandelion-labels-wordcloud." data-toc-modified-id="You-can-also-make-a-Dandelion-labels-wordcloud.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>You can also make a Dandelion labels wordcloud.</a></span></li></ul></li></ul></div>

# Visualizing Labels produced by Dandelion, using fbTREX semantic API.

First of all we load the usual libraries, and the output csvs of labels.py for two different users.

In this example, we use the sample dataset. You can change the paths for file1 and file2, as well as the number of top sources to get for both of the users.

Then we build a dataframe containing the n top sources per day for the two different users.

### First of all, let's combine the data the data for two users and take a look at the dataset.

In [ ]:
# import libraries
import pandas as pd
import altair as alt
alt.renderers.enable('notebook')

# configure files location and number of top labels to get.
file1 = 'sample_data/user_a_labels.csv'
file2 = 'sample_data/user_b_labels.csv'
top = 5

# load the data
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df = pd.concat([df1, df2])

df.head()

### The we group by word and get the n top labels for each user, each day.

In [ ]:
# filter out to get only top n labels

keep_list = df1.groupby('word')['count'].sum().nlargest(5).index.tolist()
df1 = df1[df1['word'].isin(keep_list)]
keep_list = df2.groupby('word')['count'].sum().nlargest(5).index.tolist()
df2 = df2[df2['word'].isin(keep_list)]

top = pd.concat([df1, df2])

### Panoramic of top labels per user.

In [ ]:
alt.Chart(top).mark_line().encode(
    x='impressionTime:T',
    y='count:Q',
    color='word:N',
    row='user:N'
).properties(
    width = 600,
    height = 450
)

### Compare how specific words have been appearing on two different users timelines.

Choose a list of words (in this example, 'Barcelona' and 'Partido Popular').
Then we show the trending of the two words on the two users profiles.

In [ ]:
words_list = ['Barcelona', 'Partido Popular']

filtered = df[df['word'].isin(words_list)]
alt.Chart(filtered).mark_line().encode(
    x='impressionTime:T',
    y='count:Q',
    color='user:N',
    row='word:N'
).properties(
    width = 600,
    height = 300
)

### At an higher level, we can also just compare the top contents of the two users Newsfeed.

In [ ]:

df1 = df1.sort_values('count', axis=0, ascending=False)
df2 = df2.sort_values('count', axis=0, ascending=False)

user1 = alt.Chart(df1).mark_bar().encode(
    x='count:Q',
    y=alt.Y(
        'word:N',
        sort=alt.SortField(
            field="count:Q",
            order="descending"
        )
    )
).properties(title=df1.user.value_counts().idxmax())

user2 = alt.Chart(df2).mark_bar().encode(
    x='count:Q',
    y=alt.Y(
        'word:N',
        sort=alt.SortField(
            field="count:Q",
            order="descending"
        )
    )
).properties(title=df2.user.value_counts().idxmax())

user1 & user2

## You can also make a Dandelion labels wordcloud.

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


venn = df1.word.str.join(sep='').reset_index()
print(venn)
venn.columns = ['date', 'words']
venn = venn.words.str.cat(sep=' ')

wordcloud = WordCloud(font_path='src/fonts/DejaVuSans.ttf',
                      relative_scaling = 1.0
                      ).generate(venn)
plt.imshow(wordcloud)
plt.axis("off")
plt.title('User '+df1.user.value_counts().idxmax())

